# Building a csv of all your PBs

Main moving parts:
- You'll need your user id so you can query for your runs (user name isn't good enough)
- I hit the "personal-bests" endpoint under users
   - If you want obsoleted runs as well, that's a different endpoint
- I used pandas to perform all the tabular manipulation
- Variables can't be embedded, which means subcategories aren't here.  _Yet_.
- Out comes a csv

# Get user id

In [1]:
import json
import pandas as pd
import srcomapi, srcomapi.datatypes as dt
api = srcomapi.SpeedrunCom()
s = api.search(srcomapi.datatypes.User, {"name": "baldnate"})
u = None
if len(s) == 1:
  u = s[0]
  print('found user', u.name)
  print('id is', u.id)
else:
  print(len(s), 'users found, expected 1')

found user baldnate
id is kj9yg2wx


# Get PBs (using curl, could be converted to requests)

... meanwhile on the command line ...

    > curl -O https://www.speedrun.com/api/v1/users/kj9yg2wx/personal-bests?embed=game,category,region,platform
      % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                     Dload  Upload   Total   Spent    Left  Speed
    100 1951k  100 1951k    0     0  1951k      0  0:00:01 --:--:--  0:00:01 3573k

In [2]:
f = open("pbs.json",'r',encoding='utf-8-sig')
nate = json.load(f)
rawdf = pd.DataFrame(nate['data'])

# "Simple" Columns

In [3]:
runsdf = pd.DataFrame()
runsdf['place'] = rawdf['place']
runsdf['runid'] = rawdf.apply(lambda x: x.run['id'], axis=1)
runsdf['gamename'] = rawdf.apply(lambda x: x.game['data']['names']['international'], axis=1)
runsdf['categoryname'] = rawdf.apply(lambda x: x.category['data']['name'], axis=1)
runsdf['time'] = rawdf.apply(lambda x: x.run['times']['primary_t'], axis=1)
runsdf['date'] = rawdf.apply(lambda x: x.run['date'], axis=1)
runsdf['video'] = rawdf.apply(lambda x: x.run['videos']['links'][0]['uri'], axis=1)
runsdf['comment'] = rawdf.apply(lambda x: str(x.run['comment']).replace('\n', ' ').replace('\r', ' ') , axis=1)

# Columns that need optional handling

In [4]:
def getRegion(x):
  if x.region['data'] == []:
    return None
  else:
    return x.region['data']['name']

runsdf['regionname'] = rawdf.apply(lambda x: getRegion(x), axis=1)


In [5]:
def getPlatform(x):
  if x.platform['data'] == []:
    return None
  else:
    return x.platform['data']['name']

runsdf['platformname'] = rawdf.apply(lambda x: getPlatform(x), axis=1)

# Dump to a csv

In [6]:
import csv
runsdf.to_csv('nateruns.csv', index=False, quoting=csv.QUOTE_NONNUMERIC, columns=['gamename', 'categoryname', 'platformname', 'regionname', 'place', 'time', 'date','video','comment'])
print('csv exported')

csv exported
